In [99]:
! pip3 install -qU guardrails-ai openai langchain_community langchain_experimental langchain-upstage sentence-transformers langchainhub langchain-chroma langchain matplotlib python-dotenv tavily-python ragas faiss-cpu tokenizers 


[notice] A new release of pip is available: 23.3.2 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [100]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25


[notice] A new release of pip is available: 23.3.2 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
# Set .env and define these:
# UPSTAGE_API_KEY from https://console.upstage.ai/
# TAVILY_API_KEY https://app.tavily.com
# NEWS_API_KEY from https://newsapi.org/

%load_ext dotenv
%dotenv


In [3]:
# langchain, 1. llm define, 2. prompt define, 3. chain, 4. chain.invoke

# 1. define your favorate llm, solar
from langchain_upstage import ChatUpstage
from pprint import pprint

llm = ChatUpstage()

# 2. define chat prompt
from langchain_core.prompts import ChatPromptTemplate

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_core.messages import AIMessage, HumanMessage


In [6]:
from langchain_upstage import (
    UpstageLayoutAnalysisLoader,
    UpstageGroundednessCheck,
    ChatUpstage,
    UpstageEmbeddings,
)
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter


layzer = UpstageLayoutAnalysisLoader("pdf/tesla_2023_10K.pdf", output_type="html")
# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
docs = layzer.load()  # or layzer.lazy_load()

In [7]:
for doc in docs:
    pprint(doc.page_content[:100])

("<p id='2' data-category='paragraph' style='font-size:14px'>id</p><br><h1 "
 "id='3' style='font-size:20p")


In [21]:
from IPython.display import display, HTML

display(HTML(docs[0].page_content[:1000]))

In [9]:
# RAG 1. load doc (done), 2. chunking, splits, 3. embeding - indexing, 4. retrieve

# 2. Chunk Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)
print("Splits:", len(splits))


Splits: 135


In [10]:
display(HTML(splits[11].page_content[:1000]))

In [14]:
%%time
# 3. Embed & indexing
vectorstore = Chroma.from_documents(documents=splits, embedding=UpstageEmbeddings(model="solar-embedding-1-large"))

CPU times: total: 1.55 s
Wall time: 45.3 s


In [22]:
# # vectorstore_sem = Chroma.from_documents(documents=semantic_splits, embedding=UpstageEmbeddings(model="solar-embedding-1-large"))
# vectorstore_sem = Chroma.from_documents(documents=splits, embedding=UpstageEmbeddings(model="solar-embedding-1-large"))

# 4. retrive
retriever_emb = vectorstore.as_retriever(search_kwargs={"k": 10})
result_docs = retriever_emb.invoke("This context is financial statement of Nvidia in 2022.")
print(len(result_docs))
print(result_docs[0].page_content[:100])

10
other than our local currencies at the balance<br>sheet date to compute the impact these changes wou


In [25]:
from langchain_core.output_parsers import StrOutputParser


In [37]:
# More general chat

rag_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question considering the history of the conversation. 
---
CONTEXT:
{context}
         """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)


from langchain_core.messages import AIMessage, HumanMessage

history = []

chain = rag_with_history_prompt | llm | StrOutputParser()
query1 = "Please summarize the risk from the context. You can categorize the risk into Two.\n 1) Operating risk \n 2) Financial risk"
result_docs = retriever_emb.invoke("Risk factor")

response1 = chain.invoke({"history": history, "context": result_docs, "input": query1})
print("RESPONSE1\n", response1)

RESPONSE1
 1) Operating Risk:
- The company may face issues with lithium-ion cells or other components manufactured at its Gigafactories, which may harm the production and profitability of its vehicle and energy storage products.
- The company faces strong competition from a growing list of established and new competitors, which could result in lower vehicle unit sales, price reductions, revenue shortfalls, loss of customers, and loss of market share.
- The company may experience delays in launching and ramping the production of its products and features, or it may be unable to control its manufacturing costs.
- The company's business may suffer if its ESG practices do not meet investor or other industry stakeholder expectations, which continue to evolve.
- The company's operations could be adversely affected by events outside of its control, such as natural disasters, wars, or health epidemics.
- The company's business may suffer if it cannot adequately fund its leasing program throug

In [38]:
# More general chat

rag_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question considering the history of the conversation. 
---
CONTEXT:
{context}
         """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

from langchain_core.messages import AIMessage, HumanMessage


history = [HumanMessage(query1), AIMessage(response1)]

chain = rag_with_history_prompt | llm | StrOutputParser()
query1 = "Please summarize the what are the important factors that manager think of in their operating activity."
result_docs = retriever_emb.invoke("the important factors that manager think of in their operating activity")

response2 = chain.invoke({"history": history, "context": result_docs, "input": query1})
print("RESPONSE2\n", response2)

RESPONSE2
 The important factors that managers should consider in their operating activities include:

1. Production and manufacturing: Ensuring the efficient and effective production of vehicles and energy storage products, including managing the supply chain for components and raw materials, controlling manufacturing costs, and maintaining product quality.
2. Competition: Monitoring the competitive landscape and adapting to changing market conditions to maintain market share and profitability.
3. Product development and innovation: Investing in research and development to introduce new products and features that meet customer needs and stay ahead of competitors.
4. ESG practices: Adhering to and improving environmental, social, and governance practices to meet stakeholder expectations and avoid reputational risk.
5. Risk management: Identifying and managing operational risks, such as natural disasters, health epidemics, and supply chain disruptions, to ensure business continuity.
6. 